# Import all the Lib. and Packages

In [13]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_yaml
import keras
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg16 import preprocess_input, decode_predictions
from datetime import datetime
from collections import Counter
import os
import numpy as np
import pandas as pd
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [14]:
# You need to have these three folders each with two subfolders for the two classes.
PATH = os.getcwd()
train_data_dir =PATH+ '/data/train'
validation_data_dir = PATH+'/data/val'


img_width = 224  # Change image size for training here
img_height = 224 # Change image size for training here

batch_size = 5 # i achieved good and fast results with this small minibatch size for training
batch_size_val = 5 # if Tensorflow throws a memory error while validating at end of epoch, decrease validation batch size her

In [15]:
# Create the model with the VGG16

vgg16 = VGG16(weights='imagenet', include_top=False,input_shape=(224, 224, 3))

x = vgg16.get_layer('block5_conv3').output
x = GlobalAveragePooling2D()(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)

modelFinal = Model(inputs=vgg16.input, outputs=x)

for layer in modelFinal.layers[:15]:
    layer.trainable = False
for layer in modelFinal.layers[15:]:
    layer.trainable = True

# for layer in vgg16.layers:
#     layer.trainable = False

modelFinal.summary()

opt=keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.000001, nesterov=False)

modelFinal.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

for i, layer in enumerate(modelFinal.layers):
   print(i, layer.name)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [16]:
# set data augmentation parameters here
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=70,
    width_shift_range=.3,
    height_shift_range=.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode="nearest")

# normalization neccessary for correct image input to VGG16
# datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

# no data augmentation for validation and test set
validgen = ImageDataGenerator(preprocessing_function=preprocess_input)

# validgen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

train_gen = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True, 
        #save_to_dir="_augmented_images/", 
        #save_prefix="aug_"
        )

val_gen = validgen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size_val,
        class_mode="categorical",
        shuffle=True)

Found 2000 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [17]:
def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: round(float(majority)/float(count), 2) for cls, count in counter.items()}

class_weights = get_class_weights(train_gen.classes)
print(class_weights)
print(train_gen.class_indices)


{0: 1.0, 1: 5.4, 2: 3.67}
{'00_nev': 0, '01_seb': 1, '03_Mel': 2}


In [18]:
epochs=50
train_samples = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)

# I stopped training automagically with EarlyStopping after 3 consecutive epochs without improvement
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

Intermediate_model='VGG19Inter'
Intermediate_SaveweightName= Intermediate_model+'.hdf5'
Intermediate_SaveModelName=Intermediate_model+'.yaml'

checkpoint_Intermediate = ModelCheckpoint(Intermediate_SaveweightName, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model_yaml = modelFinal.to_yaml()
with open(Intermediate_SaveModelName, "w") as yaml_file:
    yaml_file.write(model_yaml)
    
modelFinal.fit_generator(train_gen, epochs=epochs, 
                          steps_per_epoch=int(train_samples/batch_size), 
                          validation_data=val_gen, 
                          validation_steps=int(validation_samples/batch_size_val),
                          verbose=1, callbacks=[early_stopping,checkpoint_Intermediate])

Epoch 1/50
400/400 [==============================] - 222s 555ms/step - loss: 2.8726 - acc: 0.5270 - val_loss: 2.6280 - val_acc: 0.4600

Epoch 00001: val_acc improved from -inf to 0.46000, saving model to VGG19Inter.hdf5
Epoch 2/50
400/400 [==============================] - 212s 529ms/step - loss: 1.7111 - acc: 0.5610 - val_loss: 1.3465 - val_acc: 0.4800

Epoch 00002: val_acc improved from 0.46000 to 0.48000, saving model to VGG19Inter.hdf5
Epoch 3/50
400/400 [==============================] - 211s 528ms/step - loss: 1.1220 - acc: 0.6050 - val_loss: 1.0533 - val_acc: 0.5267

Epoch 00003: val_acc improved from 0.48000 to 0.52667, saving model to VGG19Inter.hdf5
Epoch 4/50
400/400 [==============================] - 210s 525ms/step - loss: 0.9348 - acc: 0.6660 - val_loss: 1.0623 - val_acc: 0.5200

Epoch 00004: val_acc did not improve from 0.52667
Epoch 5/50
400/400 [==============================] - 209s 523ms/step - loss: 0.8814 - acc: 0.6780 - val_loss: 1.1352 - val_acc: 0.5200

Epoch 0